In [3]:
# Download UrbanSound8K dataset (if not already available)
!wget https://zenodo.org/record/1203745/files/UrbanSound8K.tar.gz



--2025-09-07 17:44:20--  https://zenodo.org/record/1203745/files/UrbanSound8K.tar.gz
Resolving zenodo.org (zenodo.org)... 188.185.45.92, 188.185.43.25, 188.185.48.194, ...
Connecting to zenodo.org (zenodo.org)|188.185.45.92|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/1203745/files/UrbanSound8K.tar.gz [following]
--2025-09-07 17:44:21--  https://zenodo.org/records/1203745/files/UrbanSound8K.tar.gz
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 6023741708 (5.6G) [application/octet-stream]
Saving to: ‘UrbanSound8K.tar.gz.1’

UrbanSound8K.tar.gz 100%[===================>]   5.61G  20.3MB/s    in 5m 30s  

2025-09-07 17:49:51 (17.4 MB/s) - ‘UrbanSound8K.tar.gz.1’ saved [6023741708/6023741708]



In [4]:
!tar -xvzf UrbanSound8K.tar.gz -C /content/

Streaming output truncated to the last 5000 lines.
UrbanSound8K/audio/fold4/17480-2-0-6.wav
UrbanSound8K/audio/fold4/17480-2-0-9.wav
UrbanSound8K/audio/fold4/175904-2-0-11.wav
UrbanSound8K/audio/fold4/175904-2-0-24.wav
UrbanSound8K/audio/fold4/176003-1-0-0.wav
UrbanSound8K/audio/fold4/176638-5-0-0.wav
UrbanSound8K/audio/fold4/177756-2-0-10.wav
UrbanSound8K/audio/fold4/177756-2-0-4.wav
UrbanSound8K/audio/fold4/177756-2-0-5.wav
UrbanSound8K/audio/fold4/177756-2-0-7.wav
UrbanSound8K/audio/fold4/179862-1-0-0.wav
UrbanSound8K/audio/fold4/180977-3-1-1.wav
UrbanSound8K/audio/fold4/180977-3-1-5.wav
UrbanSound8K/audio/fold4/183989-3-1-21.wav
UrbanSound8K/audio/fold4/183989-3-1-23.wav
UrbanSound8K/audio/fold4/185709-0-0-0.wav
UrbanSound8K/audio/fold4/185709-0-0-1.wav
UrbanSound8K/audio/fold4/185709-0-0-6.wav
UrbanSound8K/audio/fold4/185709-0-0-7.wav
UrbanSound8K/audio/fold4/185909-2-0-102.wav
UrbanSound8K/audio/fold4/185909-2-0-13.wav
UrbanSound8K/audio/fold4/185909-2-0-17.wav
UrbanSound8K/audio

In [5]:
!ls /content/UrbanSound8K

audio  FREESOUNDCREDITS.txt  metadata  UrbanSound8K_README.txt


In [6]:
# -------------------------------
# 📥 Setup UrbanSound8K Dataset
# -------------------------------
import os

if not os.path.exists("/content/UrbanSound8K"):
    !wget -q https://zenodo.org/record/1203745/files/UrbanSound8K.tar.gz
    !tar -xvzf UrbanSound8K.tar.gz -C /content/
    print("✅ UrbanSound8K dataset downloaded and extracted.")
else:
    print("📂 UrbanSound8K already available.")

# Verify structure
!ls /content/UrbanSound8K


📂 UrbanSound8K already available.
audio  FREESOUNDCREDITS.txt  metadata  UrbanSound8K_README.txt


In [8]:
import pandas as pd

metadata = pd.read_csv("/content/UrbanSound8K/metadata/UrbanSound8K.csv")
metadata.head()


slice_file_name    fsID  start        end  salience  fold  classID  \
0    100032-3-0-0.wav  100032    0.0   0.317551         1     5        3   
1  100263-2-0-117.wav  100263   58.5  62.500000         1     5        2   
2  100263-2-0-121.wav  100263   60.5  64.500000         1     5        2   
3  100263-2-0-126.wav  100263   63.0  67.000000         1     5        2   
4  100263-2-0-137.wav  100263   68.5  72.500000         1     5        2   

              class  
0          dog_bark  
1  children_playing  
2  children_playing  
3  children_playing  
4  children_playing

In [11]:
import librosa
import numpy as np

def extract_features(file_path):
    try:
        audio, sr = librosa.load(file_path, duration=4)
        mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
        mfccs_scaled = np.mean(mfccs.T, axis=0)
        return mfccs_scaled
    except Exception as e:
        print(f"Error extracting {file_path}: {e}")
        return None


In [12]:
features = []

for index, row in metadata.iterrows():
    file_name = f"/content/UrbanSound8K/audio/fold{row['fold']}/{row['slice_file_name']}"
    class_label = row["classID"]

    data = extract_features(file_name)
    if data is not None:
        features.append([data, class_label])

df = pd.DataFrame(features, columns=["feature", "class_label"])
df.head()


/usr/local/lib/python3.12/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1323
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1103
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1523
  warnings.warn(


feature  class_label
0  [-211.93698, 62.58121, -122.813156, -60.745293...            3
1  [-417.0052, 99.336624, -42.995583, 51.073326, ...            2
2  [-452.39316, 112.36253, -37.57807, 43.195866, ...            2
3  [-406.47922, 91.1966, -25.043552, 42.78452, 11...            2
4  [-439.63873, 103.86223, -42.658783, 50.690285,...            2

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Features and labels
X = np.array(df["feature"].tolist())
y = np.array(df["class_label"].tolist())

# Encode labels
le = LabelEncoder()
y = le.fit_transform(y)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Training shape:", X_train.shape, y_train.shape)
print("Testing shape:", X_test.shape, y_test.shape)


Training shape: (6985, 40) (6985,)
Testing shape: (1747, 40) (1747,)


In [15]:
import librosa
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib


In [17]:
# Random Forest Classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Accuracy
acc = accuracy_score(y_test, y_pred)
print("✅ Training complete")
print("Test Accuracy:", acc)


✅ Training complete
Test Accuracy: 0.9026903262736119
